# Google Calendar API#
To do:
1. Make it merge any adjacent events
^For some reason it can't even read in anything besides the first event from the calendar

In [1]:
run quickstart.py

In [2]:
from gbt_scraper import *
import time

In [3]:
credentials = get_credentials()
http = credentials.authorize(httplib2.Http())
service = discovery.build('calendar', 'v3', http=http)

page_token = None
while True:
  calendar_list = service.calendarList().list(pageToken=page_token).execute()
  for calendar_list_entry in calendar_list['items']:
    print calendar_list_entry['summary']
  page_token = calendar_list.get('nextPageToken')
  if not page_token:
    break

isabelangelo@berkeley.edu
BL_GBT
GBT_test
Contacts
Holidays in United States


In [4]:
cal = calendar_list['items'][2]
cal['summary']

u'GBT_test'

In [5]:
scraper = GBTScraper(date=time.strftime("%m/%d/%Y"),days=10)
#scraper = GBTScraper(date='09/10/2016',days=2)
scraper.make()
events = scraper.btlpSchedule

In [2]:
def add_event(e1, e2): 
    date_start = e1.start[:10]
    time_start = e1.start[11:]+':00-07:00'
    date_end = e2.end[:10]
    time_end = e2.end[11:]+':00-07:00'
    
    event_start = date_start + 'T' + time_start
    event_end = date_end + 'T' + time_end
    
    event = {
      'summary': 'GBT Observing '+ e1.receivers +'band', #is it always the same receivers for 
      'location': '',#for consecutive sessions?
      'description': '',
      'start': {
        'dateTime': event_start,
        'timeZone': 'America/Los_Angeles',
      },
      'end': {
        'dateTime': event_end,
        'timeZone': 'America/Los_Angeles',
      },
      'recurrence': [
        'RRULE:FREQ=DAILY;COUNT=1'
      ],
      'attendees': [
        {'email': 'isabelangelo@berkeley.edu'},
        {'email': 'isabelangelo100@gmail.com'},
      ],
      'reminders': {
        'useDefault': False,
        'overrides': [
          {'method': 'email', 'minutes': 24 * 60},
          {'method': 'popup', 'minutes': 10},
        ],
      },
    }

    event = service.events().insert(calendarId=cal['id'], body=event).execute()
    print 'Event created: %s' % (event.get('htmlLink'))

In [33]:
def get_events(eventlist = []): #this command misses the first event for some reason
    now = datetime.datetime.utcnow() # 'Z' indicates UTC time
    then= datetime.datetime.utcnow()-datetime.timedelta(7)
    t = then.isoformat() + 'Z'
    eventsResult = service.events().list(
        calendarId=cal['id'], timeMin=t, maxResults=100, singleEvents=True,
        orderBy='startTime').execute()
    events = eventsResult.get('items', [])

    if not events:
        print('No upcoming events found.')
    for event in events:
        start = event['start'].get('dateTime', event['start'].get('date'))
        eventlist.append(start)

In [10]:
def start_time(event):
    return event.start[:10]+'T'+event.start[11:]+':00-07:00'
def end_time(event):
    return event.end[:10]+'T'+event.end[11:]+':00-07:00'

In [59]:
def check_if_exists(event):
    if str(start_time(event)) in eventlist:
        return True
    else:
        return False

In [60]:
eventlist = []
get_events(eventlist)
eventlist

[u'2016-09-22T19:15:00-07:00',
 u'2016-09-22T22:15:00-07:00',
 u'2016-09-24T01:00:00-07:00',
 u'2016-09-24T04:15:00-07:00',
 u'2016-09-27T19:00:00-07:00',
 u'2016-09-27T19:00:00-07:00',
 u'2016-09-28T16:15:00-07:00',
 u'2016-09-28T16:15:00-07:00',
 u'2016-09-29T06:45:00-07:00',
 u'2016-09-29T06:45:00-07:00',
 u'2016-09-30T00:45:00-07:00',
 u'2016-09-30T00:45:00-07:00',
 u'2016-09-30T18:45:00-07:00',
 u'2016-09-30T18:45:00-07:00']

In [61]:
if end_time(events[0]) in eventlist == False: #to account for it not working for first event
    add_event(events[0])
for i in range(len(events[1:])):
    if check_if_exists(event) == False:
        if i >= 1 and start_time(events[i]) == end_time(events[i-1]):
            None
        elif len(events) > i+1 and end_time(events[i]) != start_time(events[i+1]):
            add_event(events[i], events[i])
            print 'added 1'
        elif len(events) > i+1 and (end_time(events[i]) == start_time(events[i+1])) :
            if len(events) > i+2 and end_time(events[i+1]) == start_time(events[i+2]) :
                add_event(events[i], events[i+2])
                print 'added 2'
            else:
                add_event(events[i], events[i+1])
                print 'added 3'
#to make this universal, we can add a for loop that iterates over integers that go in place
#of 1,2 in i=1 and i=2

Event created: https://www.google.com/calendar/event?eid=bTBuMWkwMWY3YmxnY2RuZHNjdnM2bmpiOGNfMjAxNjA5MjhUMDIwMDAwWiBiZXJrZWxleS5lZHVfYmJyMHA1bWppbHZxamlxanFqMGh0OWw2dW9AZw
added 1
Event created: https://www.google.com/calendar/event?eid=YjQ0Zm5xMDNzZjZjdHZmZ291MGwxbmw4NjRfMjAxNjA5MjhUMjMxNTAwWiBiZXJrZWxleS5lZHVfYmJyMHA1bWppbHZxamlxanFqMGh0OWw2dW9AZw
added 1
Event created: https://www.google.com/calendar/event?eid=aTZraHBscnN1NzdzaTA3ZW00bTN1ODRjZXNfMjAxNjA5MjlUMTM0NTAwWiBiZXJrZWxleS5lZHVfYmJyMHA1bWppbHZxamlxanFqMGh0OWw2dW9AZw
added 3
Event created: https://www.google.com/calendar/event?eid=OWliYm03cGc5bGw2N2NsNjYzYmllamUyZ3NfMjAxNjA5MzBUMDc0NTAwWiBiZXJrZWxleS5lZHVfYmJyMHA1bWppbHZxamlxanFqMGh0OWw2dW9AZw
added 3
Event created: https://www.google.com/calendar/event?eid=NjFkYXY2MGhsbnBhMjA0bGExbjlyMW02aDBfMjAxNjEwMDFUMDE0NTAwWiBiZXJrZWxleS5lZHVfYmJyMHA1bWppbHZxamlxanFqMGh0OWw2dW9AZw
added 3


In [ ]:
if end_time[i] = start_time[i+1]:
    if end_time[i+1] = start_time[i+2]:
       add joining 3
    else:
        add joining two
if start_time[i] = end_time[i-1]:
    do nothing to avoid duplicates
else:
    just add the event
#this only works if there are two consecutive events